In [1]:
import os
from google import genai
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
response = client.models.generate_content(
    model = "gemma-3-27b-it",
    contents = "quanti anni ha cristo?"
)
print(response.text)

Questa è una domanda complessa che dipende da come la si interpreta! Ecco le diverse risposte possibili:

*   **Età storica di Gesù:** Secondo la maggior parte degli studiosi, Gesù è nato tra il 6 a.C. e il 4 a.C. e morì tra il 30 d.C. e il 36 d.C. Questo significa che visse circa **33-36 anni**.
*   **Età "spirituale" di Gesù:** Nella teologia cristiana, Gesù è considerato il Figlio di Dio, quindi eterno. In questo senso, si potrebbe dire che ha un'età infinita o che è sempre esistito.
*   **Anno corrente:** Se calcolassimo semplicemente aggiungendo gli anni trascorsi dalla sua nascita (stimata intorno al 4 a.C.), Gesù avrebbe circa **2023-2027 anni** nel 2024.

Quindi, la risposta dipende dal contesto in cui viene posta la domanda. Se si chiede l'età della persona storica Gesù, la risposta è intorno ai 33-36 anni. Se si chiede l'età di Gesù come figura religiosa, la risposta è che è eterno.


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("eriktks/conll2003")

# Access the train, validation, and test splits
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

# Print a sample
print(train_data[0])

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [3]:
# Print a sample from the training data
example = train_data[11001]
print("Example from the training data:")
print(example)

# If the dataset contains tokenized text, you can print it more clearly
print("\nTokens:", example["tokens"])
print("Labels:", example["ner_tags"])

Example from the training data:
{'id': '11001', 'tokens': ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.'], 'pos_tags': [11, 8, 15, 22, 22, 15, 22, 6, 22, 6, 22, 21, 10, 21, 15, 12, 16, 16, 22, 22, 21, 30, 38, 35, 22, 22, 22, 7], 'chunk_tags': [11, 12, 13, 11, 12, 13, 11, 0, 11, 0, 11, 12, 0, 11, 13, 11, 12, 12, 12, 12, 12, 3, 21, 13, 11, 12, 12, 0], 'ner_tags': [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]}

Tokens: ['1886', '-', 'At', 'Skeleton', 'Canyon', 'in', 'Arizona', ',', 'Geronimo', ',', 'Apache', 'chief', 'and', 'leader', 'of', 'the', 'last', 'great', 'Red', 'Indian', 'rebellion', 'finally', 'surrendered', 'to', 'General', 'Nelson', 'Miles', '.']
Labels: [0, 0, 0, 5, 6, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 1, 2, 0]


In [4]:
print("Dimensione del dataset di training:", len(train_data))
print("Dimensione del dataset di validazione:", len(validation_data))
print("Dimensione del dataset di test:", len(test_data))

Dimensione del dataset di training: 14041
Dimensione del dataset di validazione: 3250
Dimensione del dataset di test: 3453


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

In [5]:
# Seleziona un esempio dal dataset
example = train_data[100]  # Puoi cambiare l'indice per selezionare un altro esempio
tokens = example["tokens"]
print(len(tokens))

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# Prepara la richiesta per Gemini
response = client.models.generate_content(
    model="gemma-3-27b-it",
    contents=f"Sapendo gli NER tags: ('O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8), determina i Named Entity Recognition (NER) tags per la seguente frase: '{tokens}', stampa solo il numero associato al NER tag come provvisto in input, la tua risposta deve avere il formato: ner_tags: 1, 2, 3..."
)

# Stampa la risposta di Gemini
print("Input Tokens:", tokens)
print("Chatbot Response:", response.text)

print("Labels:", example["ner_tags"])

9
Input Tokens: ['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Chatbot Response: ner_tags: 1, 0, 0, 0, 0, 0, 0, 0, 0

Labels: [1, 0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
# Recupera e stampa tutti i modelli disponibili
available_models = client.models.list()
print("Modelli disponibili:")
for model in available_models:
    print(model)

Modelli disponibili:
name='models/chat-bison-001' display_name='PaLM 2 Chat (Legacy)' description='A legacy text-only model optimized for chat conversations' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=4096 output_token_limit=1024 supported_actions=['generateMessage', 'countMessageTokens']
name='models/text-bison-001' display_name='PaLM 2 (Legacy)' description='A legacy model that understands text and generates text as an output' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=8196 output_token_limit=1024 supported_actions=['generateText', 'countTextTokens', 'createTunedTextModel']
name='models/embedding-gecko-001' display_name='Embedding Gecko' description='Obtain a distributed representation of a text.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None,